In [1]:
# Step 1: Import necessary libraries
# --------------------------------------
# Import required libraries for document retrieval, reranking, and logging setup.
from sentence_transformers import CrossEncoder
import logging
import datetime
# link a path to ../financerag
import os

from financerag.rerank import CrossEncoderReranker
from financerag.retrieval import DenseRetrieval, SentenceTransformerEncoder
from financerag.tasks import FinDER, FinQABench, FinanceBench, TATQA, FinQA, ConvFinQA, MultiHiertt

import os  # Importing os to handle directory and file paths

# Setup basic logging configuration to show info level messages.
logging.basicConfig(level=logging.INFO)

/opt/conda/envs/pytorch/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Step 3: Initialize DenseRetriever model
# -------------------------------------
# Initialize the retrieval model using SentenceTransformers. This model will be responsible
# for encoding both the queries and documents into embeddings.
#
# You can replace 'intfloat/e5-large-v2' with any other model supported by SentenceTransformers.
# For example: 'intfloat/e5-large-v2', 'Linq-AI-Research/Linq-Embed-Mistral', etc.
encoder_model = SentenceTransformerEncoder(
    #model_name_or_path='intfloat/e5-large-v2',
    model_name_or_path='/home/ec2-user/FinanceRAG/notebook/fine_tuned_sentence_transformer_e5-large-v2_valanced/',
    query_prompt='query: ',
    doc_prompt='passage: ',
)
"""
import torch
torch.cuda.empty_cache()
encoder_model = SentenceTransformerEncoder(
    model_name_or_path='Linq-AI-Research/Linq-Embed-Mistral',
    #model_name_or_path='ProsusAI/finbert',
    query_prompt='query: ',
    #query_prompt = "financial question about the company's financial status: ",
    doc_prompt='passage: ',
    #doc_prompt = 'inquiry about financial metrics and performance: '
)
"""


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: /home/ec2-user/FinanceRAG/notebook/fine_tuned_sentence_transformer_e5-large-v2/


'\nimport torch\ntorch.cuda.empty_cache()\nencoder_model = SentenceTransformerEncoder(\n    model_name_or_path=\'Linq-AI-Research/Linq-Embed-Mistral\',\n    #model_name_or_path=\'ProsusAI/finbert\',\n    query_prompt=\'query: \',\n    #query_prompt = "financial question about the company\'s financial status: ",\n    doc_prompt=\'passage: \',\n    #doc_prompt = \'inquiry about financial metrics and performance: \'\n)\n'

In [3]:
# Step 5: Initialize CrossEncoder Reranker
# --------------------------------------
# The CrossEncoder model will be used to rerank the retrieved documents based on relevance.
#
# You can replace 'cross-encoder/ms-marco-MiniLM-L-12-v2' with any other model supported by CrossEncoder.
# For example: 'cross-encoder/ms-marco-TinyBERT-L-2', 'cross-encoder/stsb-roberta-large', etc.
reranker = CrossEncoderReranker(
    model=CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
    #model=CrossEncoder('cross-encoder/stsb-roberta-large')
)


INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda


In [ ]:
yyyy_mm_dd = datetime.datetime.now().strftime("%Y-%m-%d-%H")
output_dir = './results/'+yyyy_mm_dd
# make folder if not exist
os.makedirs(output_dir, exist_ok=True)

# Step 2: Initialize Multiple Tasks and Perform Steps 4-7
# -------------------------------------------------------
tasks = [FinDER, FinQABench, FinanceBench, TATQA, FinQA, ConvFinQA, MultiHiertt]

for task in tasks:
    task_name = task.__name__  # Get the name of the task
    print(f"Running task: {task_name}")
    
    # Step 2: Initialize the task
    finder_task = task()

    # Step 4: Perform retrieval
    retrieval_model = DenseRetrieval(
        model=encoder_model  # Ensure that encoder_model is defined earlier in the notebook
    )

    retrieval_result = finder_task.retrieve(
        retriever=retrieval_model
    )

    print(f"Retrieved results for {len(retrieval_result)} queries for {task_name}. Here's an example of the top 5 documents for the first query:")
    for q_id, result in retrieval_result.items():
        print(f"\nQuery ID: {q_id}")
        sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)
        for i, (doc_id, score) in enumerate(sorted_results[:5]):
            print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")
        break  # Only show the first query

    # Step 6: Perform reranking
    # -------------------------
    # Rerank the top 100 retrieved documents using the CrossEncoder model.
    reranking_result = finder_task.rerank(
        reranker=reranker,
        results=retrieval_result,
        top_k=100,  # Rerank the top 100 documents
        batch_size=64
    )

    # Print a portion of the reranking results to verify the output.
    print(f"Reranking results for {len(reranking_result)} queries. Here's an example of the top 5 documents for the first query:")

    for q_id, result in reranking_result.items():
        print(f"\nQuery ID: {q_id}")
        # Sort the result to print the top 5 document ID and its score
        sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

        for i, (doc_id, score) in enumerate(sorted_results[:5]):
            print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

        break  # Only show the first query


    # Step 7: Save results with the task-specific name
    # ------------------------------------------------ 
    task_output_dir = os.path.join(output_dir, task_name)

    # Ensure the directory exists
    os.makedirs(task_output_dir, exist_ok=True)

    # Save the results with the task-specific filename
    finder_task.save_results(output_dir=task_output_dir)

    # Confirm the results have been saved with the correct task name
    print(f"Results have been saved to {task_output_dir}/results.csv")


INFO:financerag.common.loader:Loading Corpus...


Running task: FinDER


INFO:financerag.common.loader:Loaded 13867 Documents.
INFO:financerag.common.loader:Corpus Example: {'id': 'ADBE20230004', 'title': 'ADBE OVERVIEW', 'text': 'Adobe is a global technology company with a mission to change the world through personalized digital experiences. For over four decades, Adobe’s innovations have transformed how individuals, teams, businesses, enterprises, institutions, and governments engage and interact across all types of media. Our products, services and solutions are used around the world to imagine, create, manage, deliver, measure, optimize and engage with content across surfaces and fuel digital experiences. We have a diverse user base that includes consumers, communicators, creative professionals, developers, students, small and medium businesses and enterprises. We are also empowering creators by putting the power of artificial intelligence (“AI”) in their hands, and doing so in ways we believe are responsible. Our products and services help unleash crea

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:financerag.retrieval.dense:Sorting corpus by document length...
INFO:financerag.retrieval.dense:Encoding corpus in batches... This may take a while.
INFO:financerag.retrieval.dense:Encoding batch 1/1...


...

INFO:financerag.retrieval.dense:Sorting corpus by document length...
INFO:financerag.retrieval.dense:Encoding corpus in batches... This may take a while.
INFO:financerag.retrieval.dense:Encoding batch 1/1...


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Retrieved results for 1663 queries for TATQA. Here's an example of the top 5 documents for the first query:

Query ID: q1a73c1d4
  Document 1: Document ID = d1a71726c, Score = -0.013570000417530537
  Document 2: Document ID = d1a735c44, Score = -0.013681773096323013
  Document 3: Document ID = d1b39b600, Score = -0.013695847243070602
  Document 4: Document ID = d1b355fa6, Score = -0.013698129914700985
  Document 5: Document ID = d1b350d80, Score = -0.013710126280784607


INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....


Batches:   0%|          | 0/2599 [00:00<?, ?it/s]

In [ ]:
# Rerank the top 100 retrieved documents using the CrossEncoder model.
reranking_result = finder_task.rerank(
    reranker=reranker,
    results=retrieval_result,
    top_k=100,  # Rerank the top 100 documents
    batch_size=64
)

In [ ]:
# merge all results
import pandas as pd
import glob

all_files = glob.glob(output_dir + "/*/*/results.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.columns = ['query_id', 'corpus_id']
frame.to_csv(output_dir + "/all_results.csv", index=False)

In [ ]:
#　Read the ground truth file